<a href="https://colab.research.google.com/github/Flychuban/Space-Soup/blob/main/TextSummarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 55.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 96.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 93.0 MB/s eta 0:00:00


In [2]:
!pip install git+https://github.com/suno-ai/bark.git -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 12.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 kB 15.4 MB/s eta 0:00:00


In [3]:
import requests
from transformers import pipeline
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from bs4 import BeautifulSoup
import re

In [4]:
# Load summarization model from HuggingFace
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


Get text from web page

In [80]:
from datetime import datetime
def search_urls_space_news():
  day = int(datetime.now().day)
  month= int(datetime.now().month)
  year = int(datetime.now().year)

  URL = f"https://www.google.com/search?q=space.com+latest+news&tbm=nws"
  print(URL)
  r = requests.get(URL)
  soup = BeautifulSoup(r.text, 'html.parser')
  atags = soup.find_all(['a'])
  hrefs = [link['href'] for link in atags]
  return hrefs


In [81]:
hrefs = search_urls_space_news()

https://www.google.com/search?q=space.com+latest+news&tbm=nws


In [49]:
print(hrefs)

['/?sa=X&ved=0ahUKEwiDqsPQ_M2BAxVLFjQIHeCSDiAQOwgC', '/search?q=yahoo+finance+apple&tbm=nws&sca_esv=569228905&ie=UTF-8&gbv=1&sei=FcsVZcPLHsus0PEP4KW6gAI', '/search?q=yahoo+finance+apple&sca_esv=569228905&ie=UTF-8&source=lnms&sa=X&ved=0ahUKEwiDqsPQ_M2BAxVLFjQIHeCSDiAQ_AUIBSgA', '/search?q=yahoo+finance+apple&sca_esv=569228905&ie=UTF-8&tbm=shop&source=lnms&sa=X&ved=0ahUKEwiDqsPQ_M2BAxVLFjQIHeCSDiAQ_AUIBygC', '/search?q=yahoo+finance+apple&sca_esv=569228905&ie=UTF-8&tbm=vid&source=lnms&sa=X&ved=0ahUKEwiDqsPQ_M2BAxVLFjQIHeCSDiAQ_AUICCgD', '/search?q=yahoo+finance+apple&sca_esv=569228905&ie=UTF-8&tbm=isch&source=lnms&sa=X&ved=0ahUKEwiDqsPQ_M2BAxVLFjQIHeCSDiAQ_AUICSgE', 'https://maps.google.com/maps?q=yahoo+finance+apple&um=1&ie=UTF-8&sa=X&ved=0ahUKEwiDqsPQ_M2BAxVLFjQIHeCSDiAQ_AUICigF', '/search?q=yahoo+finance+apple&sca_esv=569228905&ie=UTF-8&tbm=bks&source=lnms&sa=X&ved=0ahUKEwiDqsPQ_M2BAxVLFjQIHeCSDiAQ_AUICygG', '/advanced_search', '/search?q=yahoo+finance+apple&sca_esv=569228905&ie=UTF-8

In [50]:
exclude_list = ['maps', 'policies', 'preferences', 'accounts', 'support']

In [55]:
def strip_unwanted_urls(urls, exclude_list):
    val = []
    for url in urls:
        if 'https://' in url and not any(exclude_word in url for exclude_word in exclude_list):
            res = re.findall(r'(https?://\S+)', url)[0].split('&')[0]
            val.append(res)
    return list(set(val))

In [82]:
cleaned_urls = strip_unwanted_urls(hrefs, exclude_list)
cleaned_urls

['https://www.space.com/james-webb-space-telescope-evolved-galaxy-early-universe',
 'https://www.space.com/spacex-starlink-launch-group-7-3',
 'https://www.space.com/osiris-rex-asteroid-sample-caniser-lid-lifted-photos',
 'https://www.tweaktown.com/news/93488/scientists-gasped-after-they-opened-the-lid-of-nasas-asteroid-sample-capsule/index.html',
 'https://www.space.com/artemis-2-moon-mission-space-launch-system-engines-core-stage',
 'https://www.google.com/search?q%3Dspace.com%2Blatest%2Bnews%26tbm%3Dnws%26pccc%3D1',
 'https://www.space.com/iran-launches-noor-3-imaging-satellite-september-2023',
 'https://www.space.com/harvest-full-moon-will-illuminate-the-sky-on-september-29',
 'https://www.space.com/harvest-full-moon-fourth-supermoon-2023-planet-parade',
 'https://www.space.com/gravity-affects-matter-antimatter-similarly',
 'https://www.space.com/spacex-launch-psyche-asteroid-mission-one-week']

In [83]:
def get_text_from_urls(urls):
  articles = []
  titles = []
  for url in urls:
    headers = {'User-Agent': 'Mozilla/5.0'}
    cookies = {"CONSENT": "YES+cb.20210720-07-p0.en+FX+410"}
    r = requests.get(url, headers=headers, cookies=cookies)
    if r.status_code != 200:
      continue

    soup = BeautifulSoup(r.text, 'html.parser')
    paragraphs = soup.find_all(['p'])
    text = [paragraph.text for paragraph in paragraphs]
    # words = ' '.join(text)
    article = ' '.join(text)
    article = article.rstrip()

    # Minimal number of symbols in article, prevent returning cookies or terms error
    if len(article) < 200:
      continue

    articles.append(article)
    url_h1_tags = soup.find_all(['h1'])
    url_titles = [title.text.strip() for title in url_h1_tags]
    titles.append(url_titles)

  return titles, articles

In [84]:
titles, articles = get_text_from_urls(cleaned_urls)

In [85]:
articles[0]

'When you purchase through links on our site, we may earn an affiliate commission. Here’s how it works. "Based on our results astronomers must rethink our understanding of the formation of the first galaxies and how galaxy evolution occurred over the past 10 billion years." What did galaxies in the early universe look like? Surprisingly close to our own Milky Way, according to the latest findings from the James Webb Space Telescope (JWST), whose unprecedented infrared eye has been rewriting what we thought we knew about the early universe.\xa0 Astronomers have long thought that newly minted galaxies that began merging together \xa0just after the Big Bang, about 13.7 billion years ago, were too fragile to boast any noticeable structures like spiral arms, bars or rings. Those galactic features were thought to form during a time at least six billion years after the Big Bang. According to the new study, however, these delicate shapes could\'ve manifested as early as 3.7 billion years after

Split text from the article into chunks

In [86]:
sentences = []
for i, article in enumerate(articles):
  article = article.replace('.', '.<eos>')
  article = article.replace('!', '!<eos>')
  article = article.replace('?', '?<eos>')
  sentences.append(article.split('<eos>'))
sentences[-1]

['When you purchase through links on our site, we may earn an affiliate commission.',
 ' Here’s how it works.',
 " The liftoff, the eighth for SpaceX's Falcon Heavy rocket, is scheduled for Oct.",
 ' 5.',
 " We're just a week away from the first-ever interplanetary mission for SpaceX's Falcon Heavy rocket.",
 "\xa0 The Falcon Heavy is scheduled to launch NASA's Psyche spacecraft from the agency's Kennedy Space Center in Florida on Oct.",
 ' 5.',
 ' It will be the eighth mission overall for the powerful SpaceX launcher, which debuted in February 2018, and its first for NASA.',
 " Related: NASA's Psyche asteroid probe on track for October launch after 1-year delay The launch window extends through Oct.",
 ' 25, and NASA and SpaceX may need the margin given the looming possibility of a government shutdown on Oct.',
 ' 1.',
 '\xa0 If the government does shut down, NASA will likely seek a waiver to proceed with the launch, citing it as an essential operation.',
 " But there's no guarantee t

In [87]:
max_tokens_chunk = 500
chunks = []

for i, article_sentences in enumerate(sentences):
  chunks.append([])
  current_chunk = 0
  for sentence in article_sentences:
    if len(chunks[i]) == current_chunk + 1:
      if len(chunks[i][current_chunk]) + len(sentence.split(' ')) <= max_tokens_chunk:
        chunks[i][current_chunk].extend(sentence.split(' '))
      else:
        current_chunk += 1
        chunks[i].append(sentence.split(' '))
    else:
      print(current_chunk)
      chunks[i].append(sentence.split(' '))

0
0
0
0
0
0
0
0
0
0


In [88]:
joined_chunks = []

for i, article_chunks in enumerate(chunks):
    article_text = []

    for chunk_id in range(len(article_chunks)):
        article_text.append(' '.join(article_chunks[chunk_id]))

    joined_chunks.append(article_text)

In [89]:
joined_chunks[0][1]

'\xa0 —James Webb Space Telescope could detect life on Earth from across the galaxy, new study suggests —James Webb telescope detects the earliest strand in the \'cosmic web\' ever seen —\xa0James Webb Space Telescope glimpses Earendel, the most distant star known in the universe "For over 30 years it was thought that these disk galaxies were rare in the early universe due to the common violent encounters that galaxies undergo," Leonardo Ferreira, an astrophysicist at the University of Victoria in Canada and the lead author of the new study, said in the same statement.  "The fact that JWST finds so many is another sign of the power of this instrument and that the structures of galaxies form earlier in the Universe, much earlier in fact, than anyone had anticipated. " According to the new study, the team classified the sample set of close to 4,000 galaxies from the early universe by shape — like disks, point sources and spheroids.  Team members further classified them as smooth or struc

Summarize all chunks

In [91]:
def summarize_chunks(chunks):
  summarized_texts = []
  for article_chunks in chunks:
    print(len(article_chunks))
    article_text = []
    result = summarizer(article_chunks, max_length=120, min_length=30, do_sample=False)
    article_text = ' '.join([summ['summary_text'] for summ in result])
    summarized_texts.append(article_text)
  return summarized_texts

In [92]:
summarized_texts = summarize_chunks(joined_chunks)

2
2
2
1
2
2
3
3
3
2


In [93]:
summarized_texts[4]

' Four RS-25 engines have been installed on the core stage of the SLS rocket that will fly NASA\'s Artemis 2 moon mission . The rocket is scheduled to launch four astronauts around the moon in late 2024 . Artemis 2 will send four astronauts and their Orion spacecraft on a round-the-moon mission . A follow-up mission called Artemis 3 is slated to land astronauts on the moon sometime in 2026 .  Elizabeth Howell is a staff writer in the spaceflight channel since 2022 covering diversity, education and gaming . She was contributing writer for Space. com for 10 years before joining full-time . Her latest book, "Why Am I Taller?", is co-written with astronaut Dave Williams .'

In [94]:
all_summarized_text = ""
for summarized_text in summarized_texts:
  all_summarized_text += summarized_text

In [95]:
all_summarized_text

' Astronomers have long thought that newly minted galaxies that began merging together just after the Big Bang, about 13. 7 billion years ago, were too fragile to boast any noticeable structures like spiral arms, bars or rings . The new findings were based on an analysis of 3,956 galaxies, which astronomers say is the biggest sample that has been studied with JWST data .  James Webb Space Telescope could detect life on Earth from across the galaxy, new study suggests . The findings suggest a need for new ideas that explain how galaxies evolved over the past 10 billion years . The research is described in a paper published Sept. 22 in The Astrophysical Journal . A SpaceX Falcon 9 rocket launched from California early Monday morning (Sept. 25) carrying 21 Starlink satellites to orbit . The launch was streamed live via SpaceX\'s account on X (formerly Twitter) The Falcon 9\'s first stage returned to Earth safely, landing on a SpaceX drone ship at sea about 8. 5 minutes after launch .  Ast

Create a news article with all of the summarized texts

In [96]:
# model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")
# tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")

In [97]:
# summarized_texts

[' Astronomers have long thought that newly minted galaxies that began merging together just after the Big Bang, about 13. 7 billion years ago, were too fragile to boast any noticeable structures like spiral arms, bars or rings . The new findings were based on an analysis of 3,956 galaxies, which astronomers say is the biggest sample that has been studied with JWST data .  James Webb Space Telescope could detect life on Earth from across the galaxy, new study suggests . The findings suggest a need for new ideas that explain how galaxies evolved over the past 10 billion years . The research is described in a paper published Sept. 22 in The Astrophysical Journal .',
 " A SpaceX Falcon 9 rocket launched from California early Monday morning (Sept. 25) carrying 21 Starlink satellites to orbit . The launch was streamed live via SpaceX's account on X (formerly Twitter) The Falcon 9's first stage returned to Earth safely, landing on a SpaceX drone ship at sea about 8. 5 minutes after launch . 

In [98]:
# def make_news_summary(all_chunks):
#   summary_space_news = ""
#   for chunk in all_chunks:
#     inputs = tokenizer(f"Make this text like it will be used in space news report: {chunk}", return_tensors='pt', truncation=True, padding=True)
#     outputs = model.generate(**inputs, max_length=512, min_length=len(inputs['input_ids'][0]), temperature=0.25)

#     # Convert the generated output tensor to a string
#     summary_text = tokenizer.decode(outputs[0], skip_special_tokens=True, max_length=512)
#     print(outputs)
#     summary_space_news += summary_text + '\n'
#   return summary_space_news

In [99]:
# summary_space_news = make_news_summary(summarized_texts)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.25` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


tensor([[    0, 15001,  3114,   277,    43,   307,   816,    24,  6164,     3,
          8215,    26,  7466,  8606,    15,     7,    24,  1553, 12147,    53,
           544,   131,   227,     8,  2734, 19330,     6,    81,  8808,   489,
          2108,   203,   977,     6,   130,   396, 18659,    12,  8125,   136,
         20815,  5278,   114, 17723,  6026,     6,  6448,    42, 10631,     5,
            37,   126,  7469,   130,     3,   390,    30,    46,  1693,    13,
          6180,  3301,   948,  7466,  8606,    15,     7,     6,    84,     3,
         12466,   277,   497,    19,     8,  2630,  3106,    24,    65,   118,
          7463,    28,   446,   518,  4209,   331,     5,    37,  7469,  3130,
             3,     9,   174,    21,   126,   912,    24,  3209,   149,  7466,
          8606,    15,     7, 14021,   147,     8,   657,   335,  2108,   203,
             5,    37,  7469,   130,     3,   390,    30,    46,  1693,    13,
          6180,  3301,   948,  7466,  8606,    15,  

In [100]:
#summary_space_news

'Astronomers have long thought that newly minted galaxies that began merging together just after the Big Bang, about 13. 7 billion years ago, were too fragile to boast any noticeable structures like spiral arms, bars or rings. The new findings were based on an analysis of 3,956 galaxies, which astronomers say is the biggest sample that has been studied with JWST data. The findings suggest a need for new ideas that explain how galaxies evolved over the past 10 billion years. The findings were based on an analysis of 3,956 galaxies, which astronomers say is the biggest sample that has been studied with JWST data. The findings suggest a need for new ideas that explain how galaxies evolved over the past 10 billion years. The findings were based on an analysis of 3,956 galaxies, which astronomers say is the biggest sample that has been studied with JWST data. The findings suggest a need for new ideas that explain how galaxies evolved over the past 10 billion years.\nSpaceX launches rocket c

NLP for news reading

In [101]:
from bark import SAMPLE_RATE, generate_audio, preload_models
from IPython.display import Audio

In [102]:
preload_models()

Downloading: "https://dl.fbaipublicfiles.com/encodec/v0/encodec_24khz-d7cc33bc.th" to /root/.cache/torch/hub/checkpoints/encodec_24khz-d7cc33bc.th
100%|██████████| 88.9M/88.9M [00:01<00:00, 47.4MB/s]


Text to chunks and then to tts
Then append audio array

In [104]:
audio_array = generate_audio(all_summarized_text, history_prompt="v2/en_speaker_1")
Audio(audio_array, rate=SAMPLE_RATE)

Token indices sequence length is longer than the specified maximum sequence length for this model (1611 > 512). Running this sequence through the model will result in indexing errors
100%|██████████| 31/31 [00:29<00:00,  1.06it/s]
